<a href="https://colab.research.google.com/github/Itziar23/prueba/blob/master/ACNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import os
from google.colab import drive
drive.mount('/content/gdrive')
ruta='gdrive/My Drive/dataset/CAMUS/'

In [ ]:
# Standard
import os
import sys
import datetime
import numpy as np
import matplotlib.pyplot as plt
# Utils
import h5py
# Deep Learning
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

In [ ]:
PROJECT_ROOT_DIR = "/content/gdrive/MyDrive/Colab Notebooks/"
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")

In [ ]:
pip install SimpleITK

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SimpleITK as sitk
import os

%matplotlib inline

In [ ]:
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
print(os.getcwd())

In [ ]:
import sys  
sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks/architectures')
from laddernet import LadderNet
from unet import UNet
from resunet import ResUNet

In [ ]:
import sys  
sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks/metrics')
from multiclass_dice import multiclass_dice

In [ ]:
import sys  
sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks/metrics')
from iou import iou

In [ ]:
train_path = ruta + "training/patient0001/"

In [ ]:
def mhd_to_array(path):
    """
    Read an *.mhd file stored in path and return it as a numpy array.
    """
    return sitk.GetArrayFromImage(sitk.ReadImage(path, sitk.sitkFloat32))

In [ ]:
image = sitk.GetArrayFromImage(sitk.ReadImage(train_path + "patient0001_4CH_ED.mhd", sitk.sitkFloat32))
mask = sitk.GetArrayFromImage(sitk.ReadImage(train_path + "patient0001_4CH_ED_gt.mhd", sitk.sitkFloat32))

In [ ]:
earlystop = EarlyStopping(monitor='val_multiclass_dice', min_delta=0, patience=5,
                          verbose=1, mode="max", restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor='val_multiclass_dice', factor=0.2, patience=2,
                              verbose=1, mode="max", min_lr=1e-5)

In [ ]:
f = h5py.File("gdrive/My Drive/dataset/image_dataset.hdf5", "r")

In [ ]:
train_frames = f["train_4ch_ed_frames"][:,:,:,:]
train_masks = f["train_4ch_ed_masks"][:,:,:,:]
test_frames = f["test_4ch_ed_frames"][:,:,:,:]
test_masks = f["test_4ch_ed_masks"][:,:,:,:]

In [ ]:
train_frames, valid_frames, train_masks, valid_masks = train_test_split(train_frames, train_masks)

In [ ]:
import sys  
sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks/architectures')

from acnn import ACNN

In [ ]:
model_acnn= ACNN(input_size=(384, 384, 1),num_classes=4)

In [ ]:
model_acnn.summary()

In [ ]:
model_acnn.compile(optimizer=Adam(learning_rate=1e-3), loss="sparse_categorical_crossentropy", metrics=[multiclass_dice, iou,"accuracy"])

In [ ]:
history_acnn=model_acnn.fit(x=train_frames,
                    y=train_masks,
                    validation_data=(valid_frames, valid_masks),
                    batch_size=5,
                    epochs=3,
                    callbacks=[earlystop, reduce_lr])

In [ ]:
loss_acnn_4 = model_acnn.evaluate(x=test_frames,y=test_masks)

In [ ]:
print(history_acnn.history.keys())

In [ ]:
df=pd.DataFrame(history_acnn.history)
df.plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
save_fig("keras_learning_curves_plot_1")
plt.show()

In [ ]:
df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/file_name.csv')

In [ ]:
model_acnn.save_weights("/content/gdrive/MyDrive/Colab Notebooks/models/acnn_4ch_ed.h5")

In [ ]:
loss_acnn_4 = model_acnn.evaluate(x=test_frames,
                    y=test_masks)

In [ ]:
print(type(model_acnn.history))